In [1]:
from scipy.misc import imresize, imsave, fromimage, toimage
from PIL import Image
from io import BytesIO
from imageio import imread
from skimage.transform import resize
from tqdm import tqdm

import numpy as np
import time
import argparse
import warnings
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import img_util

from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D,Deconvolution2D 
from keras.layers.convolutional import Conv2D,UpSampling2D,Cropping2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import add, concatenate
from keras.layers.core import Activation

from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import Regularizer

from keras import backend as K
from keras.preprocessing import image
from keras.engine.topology import Layer
from keras.engine import InputSpec



from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

from keras.callbacks import ModelCheckpoint  
from keras import optimizers

KeyboardInterrupt: 

In [ ]:
IMG_HEIGHT = 400
IMG_WIDTH = 400
chosen_model = 'vgg16'
# Need to pass image path
base_image_path = 'static\\img\\101.jpg'
style_image_path = 'static\\img\\des_glaneuses.jpg'

content_weight = 0.025
style_weight = 1.0
total_variation_weight = 8.5e-5

In [ ]:
# dimensions of the generated picture.
def get_ratio(image):
    img = np.asarray(Image.open(image).convert('RGB')).astype('float')
    img_WIDTH = img.shape[0]
    img_HEIGHT = img.shape[1]
    aspect_ratio = float(img_HEIGHT) / img_WIDTH
    return aspect_ratio

aspect_ratio = get_ratio(base_image_path)

def preprocess_image(image_path):
    mode = "RGB"
    img = imread(image_path, pilmode=mode)

    img = resize(img,(IMG_WIDTH, IMG_HEIGHT),anti_aliasing=True)
    
    # Convert the image to a 0-255 scale
    img = 255 * img
    # RGB -> BGR
#     img = img[:, :, ::-1]

#     img[:, :, 0] -= 103.939
#     img[:, :, 1] -= 116.779
#     img[:, :, 2] -= 123.68

    # This function do the same thing above
    img = preprocess_input(img)
    
    # Theano's image channel format -- (channels, rows, cols)
    # Tensorflow's image channel format -- (rows, cols, channels)
    if K.image_dim_ordering() == "th":
        img = img.transpose((2, 0, 1)).astype('float32')

    img = np.expand_dims(img, axis=0)
    print(img.shape)
    return img

def deprocess_image(x):
    if K.image_dim_ordering() == "th":
        x = x.reshape((3, IMG_WIDTH, IMG_HEIGHT))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((IMG_WIDTH, IMG_HEIGHT, 3))

    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # BGR -> RGB
    x = x[:, :, ::-1]

    x = np.clip(x, 0, 255).astype('uint8')
    return x

def pooling_func(x, pooltype=0):
    if pooltype == 1:
        return AveragePooling2D((2, 2), strides=(2, 2))(x)
    else:
        return MaxPooling2D((2, 2), strides=(2, 2))(x)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
TEST_1 = preprocess_image(base_image_path)
TEST_1 = deprocess_image(TEST_1)
plt.imshow(TEST_1)

In [ ]:
base_image = K.variable(preprocess_image(base_image_path))
style_image = K.variable(preprocess_image(style_image_path))

if K.image_dim_ordering() == 'th':
    combination_image = K.placeholder((1, 3, IMG_WIDTH, IMG_HEIGHT))
else:
    combination_image = K.placeholder((1, IMG_WIDTH, IMG_HEIGHT, 3))

image_tensors = [base_image, style_image, combination_image]
nb_tensors = len(image_tensors)
nb_style_images = nb_tensors - 2 # Content and Output image not considered

input_tensor = K.concatenate(image_tensors, axis=0)

if K.image_dim_ordering() == "th":
    input_shape = (3, IMG_WIDTH, IMG_HEIGHT,None)
else:
    input_shape = (None, IMG_WIDTH, IMG_HEIGHT, 3)

In [ ]:
input_layer = Input(tensor=input_tensor, batch_shape=input_shape)

In [ ]:
class InputNormalize(Layer):
    def __init__(self, **kwargs):
        super(InputNormalize, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def compute_output_shape(self,input_shape):
        return input_shape

    def call(self, x, mask=None):
        #x = (x - 127.5)/ 127.5
        return x/255.




def conv_bn_relu(nb_filter, nb_row, nb_col,stride):   
    def conv_func(x):
        x = Conv2D(nb_filter, (nb_row, nb_col), strides=stride,padding='same')(x)
        x = BatchNormalization()(x)
        #x = LeakyReLU(0.2)(x)
        x = Activation("relu")(x)
        return x
    return conv_func



#https://keunwoochoi.wordpress.com/2016/03/09/residual-networks-implementation-on-keras/
def res_conv(nb_filter, nb_row, nb_col,stride=(1,1)):
    def _res_func(x):
        identity = Cropping2D(cropping=((2,2),(2,2)))(x)

        a = Conv2D(nb_filter, (nb_row, nb_col), strides=stride, padding='valid')(x)
        a = BatchNormalization()(a)
        #a = LeakyReLU(0.2)(a)
        a = Activation("relu")(a)
        a = Conv2D(nb_filter, (nb_row, nb_col), strides=stride, padding='valid')(a)
        y = BatchNormalization()(a)

        return  add([identity, y])

    return _res_func


def dconv_bn_nolinear(nb_filter, nb_row, nb_col,stride=(2,2),activation="relu"):
    def _dconv_bn(x):
        #TODO: Deconvolution2D
        #x = Deconvolution2D(nb_filter,nb_row, nb_col, output_shape=output_shape, subsample=stride, border_mode='same')(x)
        #x = UpSampling2D(size=stride)(x)
        x = UnPooling2D(size=stride)(x)
        x = ReflectionPadding2D(padding=stride)(x)
        x = Conv2D(nb_filter, (nb_row, nb_col), padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    return _dconv_bn




class Denormalize(Layer):
    '''
    Custom layer to denormalize the final Convolution layer activations (tanh)
    Since tanh scales the output to the range (-1, 1), we add 1 to bring it to the
    range (0, 2). We then multiply it by 127.5 to scale the values to the range (0, 255)
    '''

    def __init__(self, **kwargs):
        super(Denormalize, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, x, mask=None):
        '''
        Scales the tanh output activations from previous layer (-1, 1) to the
        range (0, 255)
        '''

        return (x + 1) * 127.5

    def compute_output_shape(self,input_shape):
        return input_shape


class VGGNormalize(Layer):
    '''
    Custom layer to subtract the outputs of previous layer by 120,
    to normalize the inputs to the VGG network.
    '''

    def __init__(self, **kwargs):
        super(VGGNormalize, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, x, mask=None):
        # No exact substitute for set_subtensor in tensorflow
        # So we subtract an approximate value       
        
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]       
        x -= 120
        #img_util.preprocess_image(style_image_path, img_width, img_height)
        return x
   

    def compute_output_shape(self,input_shape):
        return input_shape




class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), dim_ordering='default', **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)

        if dim_ordering == 'default':
            dim_ordering = K.image_dim_ordering()

        self.padding = padding
        if isinstance(padding, dict):
            if set(padding.keys()) <= {'top_pad', 'bottom_pad', 'left_pad', 'right_pad'}:
                self.top_pad = padding.get('top_pad', 0)
                self.bottom_pad = padding.get('bottom_pad', 0)
                self.left_pad = padding.get('left_pad', 0)
                self.right_pad = padding.get('right_pad', 0)
            else:
                raise ValueError('Unexpected key found in `padding` dictionary. '
                                 'Keys have to be in {"top_pad", "bottom_pad", '
                                 '"left_pad", "right_pad"}.'
                                 'Found: ' + str(padding.keys()))
        else:
            padding = tuple(padding)
            if len(padding) == 2:
                self.top_pad = padding[0]
                self.bottom_pad = padding[0]
                self.left_pad = padding[1]
                self.right_pad = padding[1]
            elif len(padding) == 4:
                self.top_pad = padding[0]
                self.bottom_pad = padding[1]
                self.left_pad = padding[2]
                self.right_pad = padding[3]
            else:
                raise TypeError('`padding` should be tuple of int '
                                'of length 2 or 4, or dict. '
                                'Found: ' + str(padding))

        if dim_ordering not in {'tf'}:
            raise ValueError('dim_ordering must be in {tf}.')
        self.dim_ordering = dim_ordering
        self.input_spec = [InputSpec(ndim=4)] 


    def call(self, x, mask=None):
        top_pad=self.top_pad
        bottom_pad=self.bottom_pad
        left_pad=self.left_pad
        right_pad=self.right_pad        
        
        paddings = [[0,0],[left_pad,right_pad],[top_pad,bottom_pad],[0,0]]

        
        return tf.pad(x,paddings, mode='REFLECT', name=None)

    def compute_output_shape(self,input_shape):
        if self.dim_ordering == 'tf':
            rows = input_shape[1] + self.top_pad + self.bottom_pad if input_shape[1] is not None else None
            cols = input_shape[2] + self.left_pad + self.right_pad if input_shape[2] is not None else None

            return (input_shape[0],
                    rows,
                    cols,
                    input_shape[3])
        else:
            raise ValueError('Invalid dim_ordering:', self.dim_ordering)
            
    def get_config(self):
        config = {'padding': self.padding}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))     
    
    
class UnPooling2D(UpSampling2D):
    def __init__(self, size=(2, 2)):
        super(UnPooling2D, self).__init__(size)

  
    def call(self, x, mask=None):
        shapes = x.get_shape().as_list() 
        w = self.size[0] * shapes[1]
        h = self.size[1] * shapes[2]
        return tf.image.resize_nearest_neighbor(x, (w,h))

        

class InstanceNormalize(Layer):
    def __init__(self, **kwargs):
        super(InstanceNormalize, self).__init__(**kwargs)
        self.epsilon = 1e-3
            

    def call(self, x, mask=None):
        mean, var = tf.nn.moments(x, [1, 2], keep_dims=True)
        return tf.div(tf.subtract(x, mean), tf.sqrt(tf.add(var, self.epsilon)))

                                                 
    def compute_output_shape(self,input_shape):
        return input_shape


## Loss Function

In [ ]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_dim_ordering() == 'th':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))

    shape = K.shape(x)
    
    C, W, H = (shape[0],shape[1], shape[2])
    
    cf = K.reshape(features ,(C,-1))
    gram = K.dot(cf, K.transpose(cf)) /  K.cast(C*W*H,dtype='float32')

    return gram

def style_loss(style, combination):
        assert K.ndim(style) == 3
        assert K.ndim(combination) == 3

        S = gram_matrix(style)
        C = gram_matrix(combination)
        channels = 3
        size = IMG_WIDTH * IMG_HEIGHT
        return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):

    return 0.5 * K.sum(K.square(content_weight * (combination - base)))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_dim_ordering() == 'th':
        a = K.square(x[:, :, :IMG_WIDTH - 1, :IMG_HEIGHT - 1] - x[:, :, 1:, :IMG_HEIGHT - 1])
        b = K.square(x[:, :, :IMG_WIDTH - 1, :IMG_HEIGHT - 1] - x[:, :, :IMG_WIDTH - 1, 1:])
    else:
        a = K.square(x[:, :IMG_WIDTH - 1, :IMG_HEIGHT - 1, :] - x[:, 1:, :IMG_HEIGHT - 1, :])
        b = K.square(x[:, :IMG_WIDTH - 1, :IMG_HEIGHT - 1, :] - x[:, :IMG_WIDTH - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

class StyleReconstructionRegularizer(Regularizer):
    """ Johnson et al 2015 https://arxiv.org/abs/1603.08155 """

    def __init__(self, style_feature_target, weight=1.0):
        self.style_feature_target = style_feature_target
        self.weight = weight
        self.uses_learning_phase = False
        super(StyleReconstructionRegularizer, self).__init__()

        self.style_gram = gram_matrix(style_feature_target)

    def __call__(self, x):
        output = x.output[0] # Generated by network
        loss = self.weight *  K.sum(K.mean(K.square((self.style_gram-gram_matrix(output) )))) 

        return loss


class FeatureReconstructionRegularizer(Regularizer):
    """ Johnson et al 2015 https://arxiv.org/abs/1603.08155 """

    def __init__(self, weight=1.0):
        self.weight = weight
        self.uses_learning_phase = False
        super(FeatureReconstructionRegularizer, self).__init__()

    def __call__(self, x):
        generated = x.output[0] # Generated by network features
        content = x.output[1] # True X input features

        loss = self.weight *  K.sum(K.mean(K.square(content-generated)))
        return loss


class TVRegularizer(Regularizer):
    """ Enforces smoothness in image output. """

    def __init__(self, weight=1.0):
        self.weight = weight
        self.uses_learning_phase = False
        super(TVRegularizer, self).__init__()

    def __call__(self, x):
        assert K.ndim(x.output) == 4
        x_out = x.output
        
        shape = K.shape(x_out)
        img_width, img_height,channel = (shape[1],shape[2], shape[3])
        size = img_width * img_height * channel 
        if K.image_dim_ordering() == 'th':
            a = K.square(x_out[:, :, :img_width - 1, :img_height - 1] - x_out[:, :, 1:, :img_height - 1])
            b = K.square(x_out[:, :, :img_width - 1, :img_height - 1] - x_out[:, :, :img_width - 1, 1:])
        else:
            a = K.square(x_out[:, :img_width - 1, :img_height - 1, :] - x_out[:, 1:, :img_height - 1, :])
            b = K.square(x_out[:, :img_width - 1, :img_height - 1, :] - x_out[:, :img_width - 1, 1:, :])
        loss = self.weight * K.sum(K.pow(a + b, 1.25)) 
        return loss
    
def add_style_loss(vgg,style_image_path,vgg_layers,vgg_output_dict,img_width, img_height,weight):
    style_img = img_util.preprocess_image(style_image_path, img_width, img_height)
    print('Getting style features from VGG network.')

    style_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']

    style_layer_outputs = []

    for layer in style_layers:
        style_layer_outputs.append(vgg_output_dict[layer])

    vgg_style_func = K.function([vgg.layers[-19].input], style_layer_outputs)

    style_features = vgg_style_func([style_img])

    # Style Reconstruction Loss
    for i, layer_name in enumerate(style_layers):
        layer = vgg_layers[layer_name]

        feature_var = K.variable(value=style_features[i][0])
        style_loss = StyleReconstructionRegularizer(
                            style_feature_target=feature_var,
                            weight=weight)(layer)

        layer.add_loss(style_loss)

def add_content_loss(vgg_layers,vgg_output_dict,weight):
    # Feature Reconstruction Loss
    content_layer = 'block3_conv3'
    content_layer_output = vgg_output_dict[content_layer]

    layer = vgg_layers[content_layer]
    content_regularizer = FeatureReconstructionRegularizer(weight)(layer)
    layer.add_loss(content_regularizer)


def add_total_variation_loss(transform_output_layer,weight):
    # Total Variation Regularization
    layer = transform_output_layer  # Output layer
    tv_regularizer = TVRegularizer(weight)(layer)
    layer.add_loss(tv_regularizer)


## model

In [6]:
def image_transform_net(img_width,img_height,tv_weight=1):
    x = Input(shape=(img_width,img_height,3))
    a = InputNormalize()(x)
    a = ReflectionPadding2D(padding=(40,40),input_shape=(img_width,img_height,3))(a)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res_conv(128,3,3)(a)
    a = dconv_bn_nolinear(64,3,3)(a)
    a = dconv_bn_nolinear(32,3,3)(a)
    a = dconv_bn_nolinear(3,9,9,stride=(1,1),activation="tanh")(a)
    # Scale output to range [0, 255] via custom Denormalize layer
    y = Denormalize(name='transform_output')(a)
    
    model = Model(inputs=x, outputs=y)
    
#     if tv_weight > 0:
#         add_total_variation_loss(model.layers[-1],tv_weight)
        
    return model


In [7]:
img_transform_model = image_transform_net(IMG_WIDTH, IMG_HEIGHT)

In [8]:
def vgg_loss_net(x_in, trux_x_in, width, height, style_image_path,content_weight,style_weight):
    # Append the initial input to the FastNet input to the VGG inputs
    x = concatenate([x_in, trux_x_in], axis=0)
    
    # Normalize the inputs via custom VGG Normalization layer
    x = VGGNormalize(name="vgg_normalize")(x)

    vgg = VGG16(include_top=False,input_tensor=x)

    vgg_output_dict = dict([(layer.name, layer.output) for layer in vgg.layers[-18:]])
    vgg_layers = dict([(layer.name, layer) for layer in vgg.layers[-18:]])

    if style_weight > 0:
        add_style_loss(vgg,style_image_path , vgg_layers, vgg_output_dict, width, height,style_weight)   

    if content_weight > 0:
        add_content_loss(vgg_layers,vgg_output_dict,content_weight)

    # Freeze all VGG layers
    for layer in vgg.layers[-19:]:
        layer.trainable = False

    return vgg

vgg16_model = vgg_loss_net(img_transform_model.output,img_transform_model.input,IMG_WIDTH, IMG_HEIGHT, style_image_path, content_weight, style_weight)

Getting style features from VGG network.


In [27]:
vgg16_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
input_normalize_1 (InputNormali (None, 400, 400, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
reflection_padding2d_1 (Reflect (None, 480, 480, 3)  0           input_normalize_1[0][0]          
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 480, 480, 32) 7808        reflection_padding2d_1[0][0]     
__________________________________________________________________________________________________
batch_norm

__________________________________________________________________________________________________
block4_pool (MaxPooling2D)      (None, 25, 25, 512)  0           block4_conv3[0][0]               
__________________________________________________________________________________________________
block5_conv1 (Conv2D)           (None, 25, 25, 512)  2359808     block4_pool[0][0]                
__________________________________________________________________________________________________
block5_conv2 (Conv2D)           (None, 25, 25, 512)  2359808     block5_conv1[0][0]               
__________________________________________________________________________________________________
block5_conv3 (Conv2D)           (None, 25, 25, 512)  2359808     block5_conv2[0][0]               
__________________________________________________________________________________________________
block5_pool (MaxPooling2D)      (None, 12, 12, 512)  0           block5_conv3[0][0]               
Total para

In [ ]:
gc.collect()

In [ ]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in vgg16_model.layers])
shape_dict = dict([(layer.name, layer.output_shape) for layer in vgg16_model.layers])

In [ ]:
if chosen_model == "vgg19":
    feature_layers = ['conv1_1', 'conv1_2', 'conv2_1', 'conv2_2', 'conv3_1', 'conv3_2', 'conv3_3', 'conv3_4',
                          'conv4_1', 'conv4_2', 'conv4_3', 'conv4_4', 'conv5_1', 'conv5_2', 'conv5_3', 'conv5_4']
else:
    feature_layers = ['conv1_1', 'conv1_2', 'conv2_1', 'conv2_2', 'conv3_1', 'conv3_2', 'conv3_3',
                          'conv4_1', 'conv4_2', 'conv4_3', 'conv5_1', 'conv5_2', 'conv5_3']

feature_layers = ['block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool',
                     'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 
                     'block4_conv3','block4_pool','block5_conv1', 'block5_conv2', 'block5_conv3']


# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict['block5_conv2']
print('layer_features : ', layer_features)
base_image_features = layer_features[0, :, :, :]
print('base_image_features : ', base_image_features)
combination_features = layer_features[nb_tensors - 1, :, :, :]
print('combination_features : ', combination_features)
loss += content_weight * content_loss(base_image_features,
                                          combination_features)
print('loss : ', loss)

In [ ]:
# Improvement 2
# Use all layers for style feature extraction and reconstruction
nb_layers = len(feature_layers) - 1

channel_index = 1 if K.image_dim_ordering() == "th" else -1

In [ ]:
nb_layers

In [ ]:
# Improvement 3 : Chained Inference without blurring
for i in range(len(feature_layers) - 1):
    layer_features = outputs_dict[feature_layers[i]]
    shape = shape_dict[feature_layers[i]]
    combination_features = layer_features[nb_tensors - 1, :, :, :]
    style_reference_features = layer_features[1:nb_tensors - 1, :, :, :]

    sl1 = []
    for j in range(nb_style_images):
        sl1.append(style_loss(style_reference_features[j], combination_features))

    layer_features = outputs_dict[feature_layers[i + 1]]
    shape = shape_dict[feature_layers[i + 1]]
    combination_features = layer_features[nb_tensors - 1, :, :, :]
    style_reference_features = layer_features[1:nb_tensors - 1, :, :, :]
    sl2 = []
    for j in range(nb_style_images):
        sl2.append(style_loss(style_reference_features[j], combination_features))

    for j in range(nb_style_images):
        sl = sl1[j] - sl2[j]

        # Improvement 4
        # Geometric weighted scaling of style loss
        loss += (style_weight / (2 ** (nb_layers - (i + 1)))) * sl

In [ ]:
loss += total_variation_weight * total_variation_loss(combination_image)

In [ ]:
# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)
outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

In [ ]:
f_outputs = K.function([combination_image], outputs)

In [ ]:
for layer in vgg16_model.layers:
    if layer.name in feature_layers:
        layer.add_loss(loss)

In [ ]:
for layer in model.layers[-19:]:
    layer.trainable = False

## Train part from fast-neural style

In [9]:
def dummy_loss(y_true, y_pred ):
    return K.variable(0.0)

In [10]:
nb_epoch = 4000
train_batchsize =  1
train_image_path = "static/img/"

learning_rate = 1e-3 #1e-3
optimizer = optimizers.Adam() # Adam(lr=learning_rate,beta_1=0.99)

vgg16_model.compile(optimizer,  dummy_loss)  # Dummy loss since we are learning from regularizes

datagen = image.ImageDataGenerator()

dummy_y = np.zeros((train_batchsize,IMG_WIDTH,IMG_HEIGHT,3))

In [11]:
def display_img(x):
    img = x

#     img_ht = int(IMG_WIDTH * aspect_ratio)
#     print("Rescaling Image to (%d, %d)" % (IMG_WIDTH, img_ht))
#     img = imresize(img, (IMG_WIDTH, img_ht), interp='bilinear')
#     im = toimage(img)
    plt.imshow(x)

In [18]:
# 顯示圖片
def showimg(x):
    
    # 將張量轉回圖片的後處理
    img = deprocess_image(x.copy())
    
    # 取得原圖比例
    aspect_ratio = get_ratio(base_image_path)  
    img_ht = int(IMG_WIDTH * aspect_ratio)
    print("Rescaling Image to (%d, %d)" % (IMG_WIDTH, img_ht))
    img = imresize(img, (IMG_WIDTH, img_ht), interp='bilinear')
    im = toimage(img)
    plt.imshow(im)

In [25]:
def deprocess_image(x):
    if K.image_dim_ordering() == "th":
        x = x.reshape((3, IMG_WIDTH, IMG_HEIGHT))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((IMG_WIDTH, IMG_HEIGHT, 3))

    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # BGR -> RGB
    x = x[:, :, ::-1]

    # 將陣列的值的範圍縮回 0~255，因為處理的結果有可能出現超過這個範圍的數字
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
print(train_image_path)

In [ ]:
skip_to = 0
style = 'test'
i=0
t1 = time.time()
train_start = time.time()
for x in datagen.flow_from_directory(train_image_path, class_mode=None, batch_size=train_batchsize,
    target_size=(IMG_WIDTH, IMG_HEIGHT), shuffle=False):    
    if i > nb_epoch:
        break

    if i < skip_to:
        i+=train_batchsize
        if i % 1000 ==0:
            print("skip to: %d" % i)

        continue

    hist = vgg16_model.train_on_batch(x, dummy_y)

    if i % 50 == 0:
        print(hist,(time.time() -t1))
        t1 = time.time()

    if i % 500 == 0:
        print("epoc: ", i)
        val_x = img_transform_model.predict(x)
        
#         original_img = deprocess_image(x[0].copy())
#         img = deprocess_image(val_x[0].copy())
        
        display_img(x[0])
        display_img(val_x[0])
        vgg16_model.save_weights(style+'_weights.h5')

    i+=train_batchsize
train_stop = time.time()

In [ ]:
print((train_stop - train_start) / 3600)

## Predict part

In [36]:
def load_weights(model,file_path):
    f = h5py.File(file_path)

    layer_names = [name for name in f.attrs['layer_names']]

    for i, layer in enumerate(model.layers[:31]):
        g = f[layer_names[i]]
        weights = [g[name] for name in g.attrs['weight_names']]
        layer.set_weigh
        ts(weights)

    f.close()
    
    print('Pretrained Model weights loaded.')
    
def main():
    style= style_image_path #args.style
    #img_width = img_height =  args.image_size
    output_file = 'test' #args.output
    input_file = base_image_path #args.input
    original_color = 0 #args.original_color
    blend_alpha = 0 #args.blend
    #media_filter = args.media_filter

    aspect_ratio, x = img_util.preprocess_reflect_image(input_file, size_multiple=4)

    img_width= img_height = x.shape[1]
    net = image_transform_net(img_width, img_height)
    model = vgg_loss_net(net.output,net.input,img_width,img_height,"",0,0)

    #model.summary()

    model.compile(optimizers.Adam(),  dummy_loss)  # Dummy loss since we are learning from regularizes

    model.load_weights("weights/des_glaneuses_weights.h5",by_name=False)

    
    t1 = time.time()
    y = net.predict(x)[0]
    
    y = img_util.crop_image(y, aspect_ratio)
    
    print("process: %s" % (time.time() -t1))

    ox = img_util.crop_image(x[0], aspect_ratio)

    # y =  median_filter_all_colours(y, media_filter)

    if blend_alpha > 0:
        y = blend(ox,y,blend_alpha)


    if original_color > 0:
        y = original_colors(ox,y,original_color )

    showimg(y)

In [37]:
main()

process: 2.412416458129883


ValueError: cannot reshape array of size 2359296 into shape (400,400,3)